In [80]:
import keras
import numpy as np
import pandas as pd
from time import time
#from utils import vis
from keras.models import Sequential
from keras.layers import LSTM, Dense
#from keras.utils import to_categorical
from keras.utils.np_utils import to_categorical
from keras.callbacks import TensorBoard
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.client import device_lib
from sklearn.model_selection import train_test_split

In [81]:
# get data
data = pd.read_csv('Level_1_NonPath_Path_GN_2286.csv') 

# split data
X=data.iloc[:,1:151]
y=data.iloc[:,151].astype('category')

print(X)
print(y)

          Xc1.A      Xc1.R  ...  Grantham.Xd.29  Grantham.Xd.30
0     14.767060  10.829177  ...        0.032505        0.032749
1     11.728905   8.138424  ...        0.031626        0.032946
2      8.319776   2.773259  ...        0.031596        0.031098
3     10.255686   3.778411  ...        0.031188        0.031233
4      6.745884   2.698354  ...        0.031939        0.030060
...         ...        ...  ...             ...             ...
2281  12.828743   5.186088  ...        0.031884        0.033215
2282   6.040447   3.294790  ...        0.029505        0.027986
2283   3.997220   1.598888  ...        0.031122        0.031575
2284   3.739325   1.602568  ...        0.031110        0.031553
2285   9.393071   2.415361  ...        0.033175        0.032958

[2286 rows x 150 columns]
0       1
1       1
2       1
3       1
4       1
       ..
2281    2
2282    2
2283    2
2284    2
2285    2
Name: Class, Length: 2286, dtype: category
Categories (2, int64): [1, 2]


In [82]:
# init
n_epochs = 100
n_comps = X.shape[1]

In [83]:
# encode the classification labels
#le = LabelEncoder()
#yy = to_categorical(le.fit_transform(y))

from collections import Counter
from imblearn.over_sampling import SMOTE 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


lda = LinearDiscriminantAnalysis(n_components=5)
X_ld = lda.fit(X, y).transform(X)

le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

#sm = SMOTE(random_state=45)
#x_b, y_b = sm.fit_resample(X_ld, yy)

#print('Resampled dataset shape %s' % Counter(Y_res))

# split the dataset
x_train, x_test, y_train, y_test = train_test_split(X_ld, yy, test_size=0.25,
                                                    random_state=True,
                                                    shuffle=True)

# exapand dimensions
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)

/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:463: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(150, 2 - 1) = 1 components.
  ChangedBehaviorWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:469: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)


In [105]:
# define model
model = Sequential()
model.add(LSTM(units=n_comps, return_sequences=True,
               input_shape=(x_train.shape[1], 1),
               dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(128, activation='sigmoid', return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(64, activation='sigmoid',  dropout=0.2, recurrent_dropout=0.2))



model.add(Dense(2 ,activation='softmax'))

# print model architecture summary
print(model.summary())

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_55 (LSTM)               (None, 1, 150)            91200     
_________________________________________________________________
lstm_56 (LSTM)               (None, 1, 128)            142848    
_________________________________________________________________
lstm_57 (LSTM)               (None, 64)                49408     
_________________________________________________________________
dense_29 (Dense)             (None, 2)                 130       
Total params: 283,586
Trainable params: 283,586
Non-trainable params: 0
_________________________________________________________________
None


In [106]:
# compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [107]:
history = model.fit(x_train, y_train, epochs=n_epochs, batch_size=32,
                    validation_data=(x_test, y_test))

Epoch 1/100
54/54 [==============================] - 17s 72ms/step - loss: 0.6957 - accuracy: 0.5173 - val_loss: 0.6860 - val_accuracy: 0.5175
Epoch 2/100
54/54 [==============================] - 1s 14ms/step - loss: 0.6845 - accuracy: 0.5418 - val_loss: 0.6459 - val_accuracy: 0.6678
Epoch 3/100
54/54 [==============================] - 1s 15ms/step - loss: 0.6547 - accuracy: 0.6052 - val_loss: 0.6134 - val_accuracy: 0.6556
Epoch 4/100
54/54 [==============================] - 1s 15ms/step - loss: 0.6300 - accuracy: 0.6446 - val_loss: 0.6110 - val_accuracy: 0.6626
Epoch 5/100
54/54 [==============================] - 1s 14ms/step - loss: 0.6371 - accuracy: 0.6261 - val_loss: 0.6141 - val_accuracy: 0.6573
Epoch 6/100
54/54 [==============================] - 1s 15ms/step - loss: 0.6493 - accuracy: 0.6009 - val_loss: 0.6114 - val_accuracy: 0.6626
Epoch 7/100
54/54 [==============================] - 1s 14ms/step - loss: 0.6463 - accuracy: 0.6160 - val_loss: 0.6101 - val_accuracy: 0.6678
Epoch

In [108]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
#from sklearn.metrics import roc_auc_score
import itertools

train_pred = model.predict(x_train)
test_pred = model.predict(x_test)
print("train-acc = " + str(accuracy_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-acc = " + str(accuracy_score(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
print("train-CFR = " + str(classification_report(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-CFR = " + str(classification_report(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
#print((roc_auc_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))

train-acc = 0.6417736289381564
test-acc = 0.6800699300699301
train-CFR =               precision    recall  f1-score   support

           0       0.68      0.58      0.63       892
           1       0.61      0.71      0.65       822

    accuracy                           0.64      1714
   macro avg       0.65      0.64      0.64      1714
weighted avg       0.65      0.64      0.64      1714

test-CFR =               precision    recall  f1-score   support

           0       0.72      0.62      0.67       296
           1       0.65      0.74      0.69       276

    accuracy                           0.68       572
   macro avg       0.68      0.68      0.68       572
weighted avg       0.68      0.68      0.68       572

